In [1]:
import tensorflow as tf
import numpy as np

tf.enable_eager_execution()

In [2]:
train_images_orig = np.genfromtxt('data/MNIST/train.csv', delimiter=',', skip_header=1)
test_images_orig = np.genfromtxt('data/MNIST/test.csv', delimiter=',', skip_header=1)

In [62]:
(train_images_orig.shape, test_images_orig.shape)

((42000, 785), (28000, 784))

In [63]:
train_labels,train_images = train_images_orig[:,0].copy(), train_images_orig[:,1:].copy() # label, features

In [64]:
test_images = test_images_orig.copy()

In [65]:
(train_images.shape, train_labels.shape, test_images.shape)

((42000, 784), (42000,), (28000, 784))

In [66]:
TRAINING_SIZE = len(train_images)
TEST_SIZE = len(test_images)
IMAGE_SIZE_UNSTACKED = train_images.shape[1]

TRAINING_SIZE, TEST_SIZE, IMAGE_SIZE_UNSTACKED

(42000, 28000, 784)

In [67]:
train_images = train_images.astype(np.float32)
test_images = test_images.astype(np.float32)

train_images /= 255
test_images /=  255

In [68]:
# One-hot encoding on output/label
NUM_DIGITS = 10

train_labels  = tf.keras.utils.to_categorical(train_labels, NUM_DIGITS)
train_labels = train_labels.astype(np.float32)

In [69]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(IMAGE_SIZE_UNSTACKED,)))
model.add(tf.keras.layers.Dense(256, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dense_13 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_14 (Dense)             (None, 10)                2570      
Total params: 535,818
Trainable params: 535,818
Non-trainable params: 0
_________________________________________________________________


In [70]:
# Create a TensorFlow optimizer, rather than using the Keras version
# This is currently necessary when working in eager mode
optimizer = tf.train.RMSPropOptimizer(learning_rate=0.001)

# We will now compile and print out a summary of our model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [71]:
BATCH_SIZE=128

# Because tf.data may work with potentially **large** collections of data
# we do not shuffle the entire dataset by default
# Instead, we maintain a buffer of SHUFFLE_SIZE elements
# and sample from there.
SHUFFLE_SIZE = 10000 

# Create the dataset
dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
dataset = dataset.shuffle(SHUFFLE_SIZE)
dataset = dataset.batch(BATCH_SIZE)

In [72]:
# Instead of using model.fit, we use model.train_on_batch
# as we iterate over our dataset
EPOCHS = 10

for epoch in range(EPOCHS):
    for images, labels in dataset:
        train_loss, train_accuracy = model.train_on_batch(images, labels)

    # Here you can gather any metrics or adjust your training parameters
    print('Epoch #%d\t Loss: %.6f\tAccuracy: %.6f' % (epoch + 1, train_loss, train_accuracy))

Epoch #1	 Loss: 0.128158	Accuracy: 0.937500
Epoch #2	 Loss: 0.011408	Accuracy: 1.000000
Epoch #3	 Loss: 0.004916	Accuracy: 1.000000
Epoch #4	 Loss: 0.120130	Accuracy: 0.937500
Epoch #5	 Loss: 0.000864	Accuracy: 1.000000
Epoch #6	 Loss: 0.100982	Accuracy: 0.937500
Epoch #7	 Loss: 0.000141	Accuracy: 1.000000
Epoch #8	 Loss: 0.000226	Accuracy: 1.000000
Epoch #9	 Loss: 0.000014	Accuracy: 1.000000
Epoch #10	 Loss: 0.000004	Accuracy: 1.000000


In [73]:
import os

os.makedirs('models', exist_ok=True)
os.makedirs('submissions', exist_ok=True)

In [74]:
tf.keras.models.save_model(
    model,
    "models/digit-recognizer-TF-Keras-NN-Basic3",
    include_optimizer=False
)

In [75]:
md = tf.keras.models.load_model("model/digit-recognizer-TF-Keras-NN-Basic3", compile=False)

In [76]:
prediction = md.predict(test_images)

In [77]:
label = tf.argmax(prediction,axis=1)
image_id = np.array(np.arange(1,TEST_SIZE+1))
result = np.column_stack((image_id, label))

In [78]:
np.savetxt("submissions/digit-recognizer-TF-Keras-NN-Basic3.csv", result, fmt=('%d,%d'), delimiter=",", \
           header="ImageId,Label",comments='')

In [1]:
!kaggle competitions submit -c digit-recognizer -f output/digit-recognizer-TF-Keras-NN-Basic3.csv -m "TF/Keras NN Basic 3 with 10 epochs"

Successfully submitted to Digit Recognizer


__Kaggle Scores:__

Basic
Score: 0.97614 (10 epochs)
- model = tf.keras.Sequential()
- model.add(tf.keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(IMAGE_SIZE_UNSTACKED,)))
- model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))

Basic2
Score: 0.97971 (20 epochs)
    
- model = tf.keras.Sequential()
- model.add(tf.keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(IMAGE_SIZE_UNSTACKED,)))
- model.add(tf.keras.layers.Dense(256, activation=tf.nn.relu, input_shape=(IMAGE_SIZE_UNSTACKED,)))
- model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))

Basic 3 Score: 0.97671(10 epochs)
    
- model = tf.keras.Sequential()
- model.add(tf.keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(IMAGE_SIZE_UNSTACKED,)))
- model.add(tf.keras.layers.Dense(256, activation=tf.nn.relu, input_shape=(IMAGE_SIZE_UNSTACKED,)))
- model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))